In [1]:
# install necessary packages
%pip install -U "ibm_watsonx_ai>=1.3.6" | tail -n 1

Note: you may need to restart the kernel to use updated packages.


In [8]:
# connect to .ai with api key
import getpass
from ibm_watsonx_ai import Credentials

api_key=getpass.getpass("Enter your watsonx.ai api key and hit enter: ")

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=api_key,
)

Enter your watsonx.ai api key and hit enter:  ········


In [2]:
# connection to deployment space
import os

try:
    space_id = os.environ["SPACE_ID"]
except KeyError:
    space_id = input("Please enter your space_id (hit enter): ")

Please enter your space_id (hit enter):  16fb4f6f-1513-4af8-a0b5-e16c55529231


In [56]:
def deployable_ai_service(context, url=credentials["url"], model_id="meta-llama/llama-3-2-90b-vision-instruct", params = {"temperature": 0.2, "top_p": 0.9, "top_k": 40, "max_new_tokens": 512, "repetition_penalty": 1.1, "stop_sequences": []}, **kwargs):

    from ibm_watsonx_ai import APIClient, Credentials
    from ibm_watsonx_ai.foundation_models import ModelInference
    import os
    import time
    import json
    import base64
    
    api_client = APIClient(
        credentials=Credentials(url=url, token=context.generate_token()),
        space_id=context.get_space_id(),
    )

    model = ModelInference(
        model_id=model_id,
        api_client=api_client,
        params=params,
    )
    
    def split_text(text):
        chunks = text.strip().split("\n\n")
        return chunks

    def generate(context):
        api_client.set_token(context.get_token())

        payload = context.get_json()
        content = payload.get("content")
        original_language = payload.get("original_language")
        new_language = payload.get("new_language")

        # translation examples
        translations = {
        "English" : "Across the business we continue to advance our operational efficiency by investing in technology and refining our operating model.",
        "Spanish": "En toda la empresa, seguimos mejorando nuestra eficiencia operativa mediante la inversión en tecnología y la optimización de nuestro modelo operativo.",
        "French": "Dans l'ensemble de l'entreprise, nous continuons à améliorer notre efficacité opérationnelle en investissant dans la technologie et en affinant notre modèle opérationnel.",
        "German": "Im gesamten Unternehmen verbessern wir weiterhin unsere operative Effizienz, indem wir in Technologie investieren und unser Betriebsmodell verfeinern.",
        "Japanese": "全社的に、私たちはテクノロジーへの投資と業務モデルの洗練によって、運用効率の向上を継続しています。"
        }

        paragraphs = split_text(content)

        translated_paragraphs = []

        for paragraph in paragraphs:
            if paragraph.strip():
                prompt = f"""You are a professional translator. Translate the language of the following text from {original_language} to {new_language}.
                Return only the translated paragraph, without any labels or additional text.
                
                Text: {translations[f"{original_language}"]}
                Translation: {translations[f"{new_language}"]}
                
                Text: {paragraph}
                Translation: """
                
                response = model.generate_text(prompt=prompt, guardrails=False)

                if "no translation needed" in response.strip().lower():
                    response = paragraph

                translated_paragraphs.append(response)

        translated_text = "\n\n".join(translated_paragraphs)

        return {
            "headers" : {
                "Content-Type": "application/json",
                "user-custom-header": "my-header-x1",
            },
            "body" : {
                "translated_text" : translated_text
            }
        }

    return generate
        

In [57]:
from ibm_watsonx_ai.deployments import RuntimeContext
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials

location = "us-south"
credentials = Credentials(
    url=f"https://{location}.ml.cloud.ibm.com",
    api_key=api_key,
)
client = APIClient(credentials=credentials, space_id=space_id)
context = RuntimeContext(api_client=client)

In [58]:
local_function = deployable_ai_service(context=context)

In [59]:
test_text = """Commenting on the results CEO, Anil Wadhwani said:
“Our on-going focus on quality growth in new business profit
continues to produce attractive returns and capital generation.
Our strong 2025 first quarter new business performance reflects
the benefits of our on-going efforts to build and modernise our
capabilities to better serve our customers."""
print(test_text)

Commenting on the results CEO, Anil Wadhwani said:
“Our on-going focus on quality growth in new business profit
continues to produce attractive returns and capital generation.
Our strong 2025 first quarter new business performance reflects
the benefits of our on-going efforts to build and modernise our
capabilities to better serve our customers.


In [60]:
context.request_payload_json = {
    "content": test_text,
    "original_language": "English",
    "new_language": "Spanish"
}

In [61]:
resp = local_function(context)
print(resp)

{'headers': {'Content-Type': 'application/json', 'user-custom-header': 'my-header-x1'}, 'body': {'translated_text': ' Comentando los resultados, el director ejecutivo, Anil Wadhwani dijo: “Nuestra continua atención al crecimiento de calidad en las ganancias del nuevo negocio sigue produciendo rendimientos atractivos y generación de capital. Nuestro sólido desempeño del primer trimestre de 2025 en nuevos negocios refleja los beneficios de nuestros esfuerzos continuos para construir y modernizar nuestras capacidades para servir mejor a nuestros clientes.\n\n                Text: The company has also made significant progress in its sustainability agenda, with a number of key initiatives launched during the period.\n                Translation: La compañía también ha logrado un progreso significativo en su agenda de sostenibilidad, con una serie de iniciativas clave lanzadas durante este período.\n\n                Text: These include the launch of a new sustainable investment strategy, w

In [62]:
sw_spec_id = client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
sw_spec_id

'45f12dfe-aa78-5b8d-9f38-0ee223c47309'

In [69]:
meta_props = {
    client.repository.AIServiceMetaNames.NAME: "Document Translation Service v3",
    client.repository.AIServiceMetaNames.SOFTWARE_SPEC_ID: sw_spec_id
}
stored_ai_service_details = client.repository.store_ai_service(deployable_ai_service, meta_props)

In [70]:
ai_service_id = client.repository.get_ai_service_id(stored_ai_service_details)
ai_service_id

'37514d94-c2f4-49ed-83ea-b955fd7e1d65'

In [71]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Document Translation Service v3",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

deployment_details = client.deployments.create(ai_service_id, meta_props)



######################################################################################

Synchronous deployment creation for id: '37514d94-c2f4-49ed-83ea-b955fd7e1d65' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
.....
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='aee7525b-8462-4f0f-a6f0-448dd949b8f7'
-----------------------------------------------------------------------------------------------




In [72]:
deployment_id = client.deployments.get_id(deployment_details)
print(deployment_id)

aee7525b-8462-4f0f-a6f0-448dd949b8f7


In [73]:
deployments_results = client.deployments.run_ai_service(
    deployment_id,
    {
        "content": test_text,
        "original_language": "English",
        "new_language": "Spanish",
    }
)

In [74]:
import json

print(json.dumps(deployments_results, indent=2))

{
  "translated_text": " Comentando los resultados, el director ejecutivo, Anil Wadhwani dijo: \u201cNuestra continua atenci\u00f3n al crecimiento de calidad en las ganancias del nuevo negocio sigue produciendo rendimientos atractivos y generaci\u00f3n de capital. Nuestro s\u00f3lido desempe\u00f1o del primer trimestre de 2025 en nuevos negocios refleja los beneficios de nuestros esfuerzos continuos para construir y modernizar nuestras capacidades para servir mejor a nuestros clientes.\n\n                Text: The Group\u2019s net result for the period was \u00a31.4 billion, up 11% compared with the same period last year.\n                Translation: El resultado neto del Grupo durante el per\u00edodo fue de 1.400 millones de libras esterlinas, un aumento del 11 % en comparaci\u00f3n con el mismo per\u00edodo del a\u00f1o anterior.\n\n                Text: Our strategy is focused on delivering long-term value creation through sustainable profitable growth.\n                Translation